In [1]:
# IMPORT NEEDED LIBRARIES
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
from numpy import asarray
import rioxarray as rxr
import rasterio as rio
from rasterio.plot import plotting_extent
from rasterio.enums import Resampling
import geopandas as gpd
import scipy
from scipy.stats import gaussian_kde
from scipy.stats import stats
from scipy.stats import mode
import seaborn as sns
import xarray as xr
import pandas as pd
import glob
import os
import fnmatch
import math
import statistics as st
from math import e
#from multispectral_functions import *

In [2]:
# SET IMAGE AND SHAPEFILE DIRECTORIES

# Pre-fire & RGB directory
dirImgLoc = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/sentinel_2b/raw/S2B_MSIL1C_20171128T184719_N0206_R070_T11SKU_20171128T203222.SAFE/GRANULE/L1C_T11SKU_A003814_20171128T184714/IMG_DATA/'

# Post-fire directory
dirImgLocPost = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/sentinel_2b/raw/S2B_MSIL1C_20171228T184749_N0206_R070_T11SKU_20171228T221718.SAFE/GRANULE/L1C_T11SKU_A004243_20171228T184751/IMG_DATA/'

# Shapefile directory, ensure correct number of shapefiles for analysis
dirPolyLoc = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/shapefiles/'

# AVIRIS imagery
pre_h = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/registered_data/171206_av_area_final_clip_warp_warp_warp.dat'

post_h = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/registered_data/171221_av_area_final_warp_warp.dat'

av_dnbr = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/registered_data/dnbr_aviris_warp.dat'


# Output folders
rgb_out = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/batch_output/rgb/'
nbr_out = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/batch_output/nbr/'
dnbr_out = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/batch_output/dnbr/'
usgs_dnbr_out = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/batch_output/usgs_dnbr/'
usgs_pha_out = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/batch_output/usgs_pha'
ppp_out = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/batch_output/ppp/'
fig_out = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/batch_output/fig/'
general = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/batch_output/general/'

In [112]:
# ALL FUNCTIONS

def nbr_clip(dir, 
             shape, 
             plot=False, 
             save=False):
    '''Takes in image and clips the raster to study basins.
        
        CLIP OPTIONS:
        all_touched (boolean, OPTIONAL): If True, all pixels touched by geometries
        will be burned in. If false, only pixels whose center is within the polygon 
        or that are selected by Bresenham’s line algorithm will be burned in.
        
        drop (bool, optional): If True, drop the data outside of the extent of the 
        mask geoemtries Otherwise, it will return the same raster with the data masked. 
        Default is True.
        
        invert (boolean, optional): If False, pixels that do not overlap shapes will be
        set as nodata. Otherwise, pixels that overlap the shapes will be set as nodata. 
        False by default.'''
    
    NIR_path = glob.glob(dir + '/*B8A.jp2')
    NIR = rxr.open_rasterio(NIR_path[0], masked=True)
    SWIR_path = glob.glob(dir + '/*B12.jp2')
    SWIR = rxr.open_rasterio(SWIR_path[0], masked=True)
    
    name = os.path.basename(NIR_path[0])
    name_split = name.split('_')
    val = name_split[1]
    date = val[:8]
    sdc = f"{val[4:6]}/{val[6:8]}/{val[0:4]}"
    
    shape = gpd.read_file(os.path.join(dirPolyLoc, filename))
    b_name = filename.split('.')
    basin = b_name[0]
    basin_NIR = NIR.rio.clip(shape.geometry.values,
                                shape.crs,
                                all_touched = False,
                                drop = True,
                                invert = False)
    basin_SWIR = SWIR.rio.clip(shape.geometry.values,
                                shape.crs,
                                all_touched = False,
                                drop = True,
                                invert = False)
    
    #basin_clip.plot()
    
    
    nbr = (basin_NIR - basin_SWIR) / (basin_NIR + basin_SWIR)
    if plot == True:
        nbr.plot(
            cmap = 'gray',
            vmax = 1,
            vmin = -1)
        plt.title(f"Basin {basin} Normalized Burn Ratio: {sdc}")
   
    if save == True:
        outname = f"{date}_{basin}_nbr.tif"
        nbr.rio.to_raster(os.path.join(nbr_out, outname), driver='GTIFF')
    
    return nbr


def av_nbr_clip(image, 
             shape, 
             save=False):
    n = os.path.basename(image)
    n2 = n.split('_')
    date = n2[0]
    image = rxr.open_rasterio(image, masked=True)
    nir = image[62,:,:]
    swir = image[217,:,:]
    shape = gpd.read_file(os.path.join(dirPolyLoc, filename))
    basin_NIR = nir.rio.clip(shape.geometry.values,
                                shape.crs,
                                all_touched = False,
                                drop = True,
                                invert = False)
    
    basin_SWIR = swir.rio.clip(shape.geometry.values,
                                shape.crs,
                                all_touched = False,
                                drop = True,
                                invert = False)
    
    nbr = (basin_NIR - basin_SWIR) / (basin_NIR + basin_SWIR)
    if save == True:
        outname = f"{date}_{basin}_h_nbr.tif"
        nbr.rio.to_raster(os.path.join(nbr_out, outname), driver='GTIFF')
    return nbr
#####
def dnbr(pre_fire, 
         post_fire, 
         plot=False, 
         save=False):
    '''Takes in nbr functions outputs and calculates the differenced normalized burn ratio.
        This function outputs individual basin results.'''
    
    dnbr = pre_fire - post_fire
    
    if plot == True:
        dnbr.plot(
            cmap = 'gray',
            vmax = 1,
            vmin = -1)
        plt.title(f"Basin {basin} Differenced Normalized Burn Ratio")
        
    if save == True:
        outname = f"{basin}_dnbr.tif"
        dnbr.rio.to_raster(os.path.join(dnbr_out, outname), driver='GTIFF')
        
    return dnbr

def avg_dnbr(pre_fire, 
         post_fire, 
         plot=False, 
         save=False):
    '''Takes in nbr functions outputs and calculates the differenced normalized burn ratio.
        This function outputs individual basin results.'''
    
    dnbr = pre_fire - post_fire
    dnbr_flat = dnbr.values.flatten()
    pixarr = np.empty((dnbr_flat.shape[0]), dtype= float)
    pixarr[:] = np.nan
    x = np.where(np.isnan(dnbr_flat) == False)
    dnbrdata = dnbr_flat[x]
    dnbr_val.append(dnbrdata)
    p1 = []
    for i in dnbrdata:
        individual_basin = x_values_df.index[x_values_df['basin_id'] == int(basin)]
        x_2 = x_values_df['x2'][individual_basin]
        p1.append(x_2)
    
    # Append average array, flatten data, reshape into image
    avg_arr = np.array(p1)
    a_final = avg_arr.flatten()
    pixarr[x] = a_final
    pixarrImg = pixarr.reshape((dnbr.shape[1], dnbr.shape[2]))
    out_dnbr = xr.Dataset()
    out_dnbr = xr.DataArray(pixarrImg, dims = ('y', 'x'),
                            coords = {'x': dnbr.coords['x'],
                                      'y': dnbr.coords['y']})

    if plot == True:
        out_dnbr.plot(
            cmap = 'gray',
            vmax = 1,
            vmin = -1)
        plt.title(f"Basin {basin} USGS Differenced Normalized Burn Ratio")
        
    if save == True:
        outname = f"{basin}_usgs_dnbr.tif"
        out_dnbr.rio.to_raster(os.path.join(usgs_dnbr_out, outname), driver='GTIFF')
        
    return out_dnbr
        
def pha(dnbr, 
          plot=False, 
          save=False):
    
    # Coefficients for Southern California
    b = -3.63
    b_1 = 0.41
    b_2 = 0.67
    b_3 = 0.7
    
    
    dnbr_flat = dnbr.values.flatten()
    pixarr = np.empty((dnbr_flat.shape[0]), dtype= float)
    pixarr[:] = np.nan
    x = np.where(np.isnan(dnbr_flat) == False)
        
    # Getting the data from dnbr_flat
    dnbrdata = dnbr_flat[x]
        
    # Matching index of x_values_df[basin_id] to shapefile
    individual_basin = x_values_df.index[x_values_df['basin_id'] == int(basin)]
        
    # Pulling individual values from x_1 and x_3, creating pixel values for x_2 denoted by s & s1
    x_1 = x_values_df['x1'][individual_basin]
    x_2 = x_values_df['x2'][individual_basin]
    x_3 = x_values_df['x3'][individual_basin]
       
    # Per pixel analysis, probability stored in empty list p1
    p1 = []
    for i in dnbrdata:
        lnx =  b + (b_1 * x_1 * val) + (b_2 * x_2 * val) + (b_3 * x_3 * val)
        prob = (e ** lnx) / (1.0 + e ** lnx)
        p1.append(prob)
        
    # Append probability array, flatten data, reshape into image
    prob_arr = np.array(p1)
    p_final = prob_arr.flatten()
    pixarr[x] = p_final
    pixarrImg = pixarr.reshape((dnbr.shape[1], dnbr.shape[2]))
    out_dnbr = xr.Dataset()
    out_dnbr = xr.DataArray(pixarrImg, dims = ('y', 'x'),
                            coords = {'x': dnbr.coords['x'],
                                      'y': dnbr.coords['y']})
    
    pha_list.append(out_dnbr)
    if plot == True:
        out_dnbr.plot(
            cmap = 'gnuplot',
            vmax = 1,
            vmin = 0)
        plt.title(f"Basin {basin} Initiation Probablilty, R = {int(val*4)}mm/hr")
    
    if save == True:
        outname = f"{basin}_usgsP_{int(val*4)}mmhr.tif"
        out_dnbr.rio.to_raster(os.path.join(usgs_pha_out, outname), driver='GTIFF')
        print(f"Basin {basin} {int(val*4)}mm/hr prob analysis saved as geotiff")
        
        print(f"Basin {basin} {int(val*4)}mm/hr prob analysis complete")
    return out_dnbr

def pha_mode(dnbr, 
          plot=False, 
          save=False):
    
    # Coefficients for Southern California
    b = -3.63
    b_1 = 0.41
    b_2 = 0.67
    b_3 = 0.7
    
    
    dnbr_flat = dnbr.values.flatten()
    pixarr = np.empty((dnbr_flat.shape[0]), dtype= float)
    pixarr[:] = np.nan
    x = np.where(np.isnan(dnbr_flat) == False)
        
    # Getting the data from dnbr_flat
    dnbrdata = dnbr_flat[x]
        
    # Matching index of x_values_df[basin_id] to shapefile
    individual_basin = x_values_df.index[x_values_df['basin_id'] == int(basin)]
        
    # Pulling individual values from x_1 and x_3, creating pixel values for x_2 denoted by s & s1
    x_1 = x_values_df['x1'][individual_basin]
    x_2 = mode
    x_3 = x_values_df['x3'][individual_basin]
       
    # Per pixel analysis, probability stored in empty list p1
    p1 = []
    for i in dnbrdata:
        lnx =  b + (b_1 * x_1 * val) + (b_2 * x_2 * val) + (b_3 * x_3 * val)
        prob = (e ** lnx) / (1.0 + e ** lnx)
        p1.append(prob)
        
    # Append probability array, flatten data, reshape into image
    prob_arr = np.array(p1)
    p_final = prob_arr.flatten()
    pixarr[x] = p_final
    pixarrImg = pixarr.reshape((dnbr.shape[1], dnbr.shape[2]))
    out_dnbr = xr.Dataset()
    out_dnbr = xr.DataArray(pixarrImg, dims = ('y', 'x'),
                            coords = {'x': dnbr.coords['x'],
                                      'y': dnbr.coords['y']})
    
    pha_mode_list.append(out_dnbr)
    if plot == True:
        out_dnbr.plot(
            cmap = 'gnuplot',
            vmax = 1,
            vmin = 0)
        plt.title(f"Basin {basin} Initiation Probablilty, R = {int(val*4)}mm/hr")
    
    if save == True:
        outname = f"{basin}_usgsP_{int(val*4)}mmhr_mode.tif"
        out_dnbr.rio.to_raster(os.path.join(usgs_pha_out, outname), driver='GTIFF')
        print(f"Basin {basin} {int(val*4)}mm/hr prob analysis saved as geotiff")
        
        print(f"Basin {basin} {int(val*4)}mm/hr prob analysis complete")
    return out_dnbr

def pha_median(dnbr, 
          plot=False, 
          save=False):
    
    # Coefficients for Southern California
    b = -3.63
    b_1 = 0.41
    b_2 = 0.67
    b_3 = 0.7
    
    
    dnbr_flat = dnbr.values.flatten()
    pixarr = np.empty((dnbr_flat.shape[0]), dtype= float)
    pixarr[:] = np.nan
    x = np.where(np.isnan(dnbr_flat) == False)
        
    # Getting the data from dnbr_flat
    dnbrdata = dnbr_flat[x]
        
    # Matching index of x_values_df[basin_id] to shapefile
    individual_basin = x_values_df.index[x_values_df['basin_id'] == int(basin)]
        
    # Pulling individual values from x_1 and x_3, creating pixel values for x_2 denoted by s & s1
    x_1 = x_values_df['x1'][individual_basin]
    x_2 = median
    x_3 = x_values_df['x3'][individual_basin]
       
    # Per pixel analysis, probability stored in empty list p1
    p1 = []
    for i in dnbrdata:
        lnx =  b + (b_1 * x_1 * val) + (b_2 * x_2 * val) + (b_3 * x_3 * val)
        prob = (e ** lnx) / (1.0 + e ** lnx)
        p1.append(prob)
        
    # Append probability array, flatten data, reshape into image
    prob_arr = np.array(p1)
    p_final = prob_arr.flatten()
    pixarr[x] = p_final
    pixarrImg = pixarr.reshape((dnbr.shape[1], dnbr.shape[2]))
    out_dnbr = xr.Dataset()
    out_dnbr = xr.DataArray(pixarrImg, dims = ('y', 'x'),
                            coords = {'x': dnbr.coords['x'],
                                      'y': dnbr.coords['y']})
    
    pha_median_list.append(out_dnbr)
    if plot == True:
        out_dnbr.plot(
            cmap = 'gnuplot',
            vmax = 1,
            vmin = 0)
        plt.title(f"Basin {basin} Initiation Probablilty, R = {int(val*4)}mm/hr")
    
    if save == True:
        outname = f"{basin}_usgsP_{int(val*4)}mmhr_median.tif"
        out_dnbr.rio.to_raster(os.path.join(usgs_pha_out, outname), driver='GTIFF')
        print(f"Basin {basin} {int(val*4)}mm/hr prob analysis saved as geotiff")
        
        print(f"Basin {basin} {int(val*4)}mm/hr prob analysis complete")
    return out_dnbr

def ppp(dnbr, 
          plot=False, 
          save=False):
    
    # Coefficients for Southern California
    b = -3.63
    b_1 = 0.41
    b2 = 0.67
    b_2 = np.array(b2)
    b_3 = 0.7
    
    
    dnbr_flat = dnbr.values.flatten()
    pixarr = np.empty((dnbr_flat.shape[0]), dtype= float)
    pixarr[:] = np.nan
    x = np.where(np.isnan(dnbr_flat) == False)
        
    # Getting the data from dnbr_flat
    dnbrdata = dnbr_flat[x]
        
    # Matching index of x_values_df[basin_id] to shapefile
    individual_basin = x_values_df.index[x_values_df['basin_id'] == int(basin)]
        
    # Pulling individual values from x_1 and x_3, creating pixel values for x_2 denoted by s & s1
    x_1 = x_values_df['x1'][individual_basin]
    x_3 = x_values_df['x3'][individual_basin]
    s = np.multiply(dnbrdata, b_2)
    s1 = np.multiply(s, val)
    # s1 multiplication assumes when function is called that val is defined globally in code.
    
    # Per pixel analysis, probability stored in empty list p1
    p1 = []
    for i in s1:
        lnx =  b + (b_1 * x_1 * val) + i + (b_3 * x_3 * val)
        prob = (e ** lnx) / (1.0 + e ** lnx)
        p1.append(prob)
       
    # Append probability array, flatten data, reshape into image
    prob_arr = np.array(p1)
    p_final = prob_arr.flatten()
    dist_list.append(p_final)
    pixarr[x] = p_final
    pixarrImg = pixarr.reshape((dnbr.shape[1], dnbr.shape[2]))
    out_dnbr = xr.Dataset()
    out_dnbr = xr.DataArray(pixarrImg, dims = ('y', 'x'),
                            coords = {'x': dnbr.coords['x'],
                                      'y': dnbr.coords['y']})
    ppp_list.append(out_dnbr)
    if plot == True:
        out_dnbr.plot(
            cmap = 'gnuplot',
            vmax = 1,
            vmin = 0)
        plt.title(f"Basin {basin} Per Pixel Initiation Probablilty, R = {int(val*4)}mm/hr")
    
    if save == True:
        outname = f"{basin}_ppp_{int(val*4)}mmhr.tif"
        out_dnbr.rio.to_raster(os.path.join(ppp_out, outname), driver='GTIFF')
        print(f"Basin {basin} {int(val*4)}mm/hr per pixel analysis saved as geotiff")
        
        print(f"Basin {basin} {int(val*4)}mm/hr per pixel analysis complete")
    return out_dnbr

#####

def rgb(dir, 
        shape, 
        plot=False, 
        save=False):
    
    def normalize(array):
        """Normalizes numpy arrays into scale 0.0 - 1.0"""
        array_min, array_max = array.min(), array.max()
        return ((array - array_min)/(array_max - array_min))
    def resample(rst):
        scale_factor = 1/2
        new_width = rst.rio.width * scale_factor
        new_height = rst.rio.height * scale_factor
        sampled = rst.rio.reproject(rst.rio.crs, 
                                    shape=((int(new_height), int(new_width))),
                                    resampling = Resampling.nearest)
        return sampled
    
    # Set RGB Bands,Sentinel-2 is red(B04), green(B03), blue(B02)
    imageRed = glob.glob(dir + '/*B04.jp2')
    imageGreen = glob.glob(dir + '/*B03.jp2')
    imageBlue = glob.glob(dir + '/*B02.jp2')
    name = os.path.basename(imageRed[0])
    name_split = name.split('_')
    val = name_split[1]
    date = val[:8]
    sdc = f"{val[4:6]}/{val[6:8]}/{val[0:4]}"
    imageRed_open = rxr.open_rasterio(imageRed[0], masked=True)
    imageBlue_open = rxr.open_rasterio(imageBlue[0], masked=True)
    imageGreen_open = rxr.open_rasterio(imageGreen[0], masked=True)
    
    shape = gpd.read_file(os.path.join(dirPolyLoc, filename))
    b_name = filename.split('.')
    basin = b_name[0]
    
    redClip = imageRed_open.rio.clip(shape.geometry.values, 
                                     shape.crs, 
                                     all_touched=False, 
                                     drop=True, 
                                     invert=False)
    blueClip = imageBlue_open.rio.clip(shape.geometry.values, 
                                       shape.crs, 
                                       all_touched=False, 
                                       drop=True, 
                                       invert=False)
    greenClip = imageGreen_open.rio.clip(shape.geometry.values, 
                                         shape.crs, 
                                         all_touched=False, 
                                         drop=True, 
                                         invert=False)
    
    # Normalize clip so rgb display is bright enough
    redn = normalize(redClip)
    bluen = normalize(blueClip)
    greenn = normalize(greenClip)
    
    # Resample from 10m to 20m spatial resolution
    red_rs = resample(redn)
    blue_rs = resample(bluen)
    green_rs = resample(greenn)
    
    # Pull band information from each array
    redData = red_rs.data[0,:,:]
    blueData = blue_rs.data[0,:,:]
    greenData = green_rs.data[0,:,:]
    
    # Create and plot mask
    dataPlot = np.zeros((redData.shape[0],redData.shape[1],3))
    dataPlot[:,:,0] = redData
    dataPlot[:,:,1] = greenData
    dataPlot[:,:,2] = blueData
    dataPlotMask = np.ma.masked_where(np.isnan(dataPlot), dataPlot)
    img = xr.Dataset()
    img = xr.DataArray(dataPlotMask,
                           dims = ('y', 'x', 'band'),
                           coords ={'y': red_rs.coords['y'], 
                                    'x': red_rs.coords['x'],
                                    'band': ["red", "green", "blue"]})
    
    
    # Transpose array for geotiff output
    out_img = img.transpose('band', 'y', 'x')
    
    if plot == True:
        img.plot.imshow()
        plt.title(f'Basin {basin} RGB: {sdc}')
        plt.show()
    #Saving
    if save == True:
        outname = f"{basin}_{date}_rgb.tif"
        out_img.rio.to_raster(os.path.join(rgb_out, outname), driver='GTIFF')
    print(f"Basin {basin} RGB combine complete")
    return out_img

####

def ppp_av(dnbr, 
          plot=False, 
          save=False):
    
    # Coefficients for Southern California
    b = -3.63
    b_1 = 0.41
    b2 = 0.67
    b_2 = np.array(b2)
    b_3 = 0.7
    
    
    dnbr_flat = dnbr.values.flatten()
    pixarr = np.empty((dnbr_flat.shape[0]), dtype= float)
    pixarr[:] = np.nan
    x = np.where(np.isnan(dnbr_flat) == False)
        
    # Getting the data from dnbr_flat
    dnbrdata = dnbr_flat[x]
    print(f'dnbrdata: {dnbrdata}')   
    # Matching index of x_values_df[basin_id] to shapefile
    individual_basin = x_values_df.index[x_values_df['basin_id'] == int(basin)]
        
    # Pulling individual values from x_1 and x_3, creating pixel values for x_2 denoted by s & s1
    x_1 = x_values_df['x1'][individual_basin]
    x_3 = x_values_df['x3'][individual_basin]
    s = np.multiply(dnbrdata, b_2)
    s1 = np.multiply(s, val)
    # s1 multiplication assumes when function is called that val is defined globally in code.
    
    # Per pixel analysis, probability stored in empty list p1
    p1 = []
    for i in s1:
        lnx =  b + (b_1 * x_1 * val) + i + (b_3 * x_3 * val)
        prob = (e ** lnx) / (1.0 + e ** lnx)
        p1.append(prob)
       
    # Append probability array, flatten data, reshape into image
    prob_arr = np.array(p1)
    p_final = prob_arr.flatten()
    dist_list.append(p_final)
    pixarr[x] = p_final
    print(f'pixarr: {pixarr}')
    print(f'dnbr shape: {dnbr.shape}')
    pixarrImg = pixarr.reshape((dnbr.shape[1], dnbr.shape[2]))
    out_dnbr = xr.Dataset()
    out_dnbr = xr.DataArray(pixarrImg, dims = ('y', 'x'),
                            coords = {'x': dnbr.coords['x'],
                                      'y': dnbr.coords['y']})
    
    if plot == True:
        out_dnbr.plot(
            cmap = 'gnuplot',
            vmax = 1,
            vmin = 0)
        plt.title(f"Basin {basin} Per Pixel Initiation Probablilty, R = {int(val*4)}mm/hr")
    
    if save == True:
        outname = f"{basin}_ppp_{int(val*4)}mmhr.tif"
        out_dnbr.rio.to_raster(os.path.join(ppp_out, outname), driver='GTIFF')
        print(f"Basin {basin} {int(val*4)}mm/hr per pixel analysis saved as geotiff")
        
        print(f"Basin {basin} {int(val*4)}mm/hr per pixel analysis complete")
    return out_dnbr

####
def dnbr_hist(data):
    avg = np.mean(data)
    fig, ax = plt.subplots()
        
    data.plot.hist(ax=ax,
                   color='c',
                   edgecolor='k',
                   bins=20)
    plt.axvline(avg, color='k', linestyle='dashed', linewidth=2)
    min_ylim, max_ylim = plt.ylim()
    plt.text(avg*1.05, max_ylim*0.95, 'Mean: {:.3f}'.format(avg))   
    ax.set(
        title = f"Basin {basin} dNBR Distribution",
        xlabel='dNBR',
        ylabel='Frequency')
    plt.show()
    return fig



#####



def fig_1x3(tci, 
             bs, 
             prob,
             savefig=False):
    # Create figures
    fig, ax = plt.subplots(1, 3, 
                           figsize=(14,4), 
                           gridspec_kw={'width_ratios':[1, 1.2, 1.2]})
    fig.suptitle(f'Basin {basin} with {int(val*4)}mm/hr Design Storm', fontsize=15, fontweight='bold')
    
    # RGB imagery
    tci.plot.imshow(ax=ax[0])
    ax[0].axis('off')
    ax[0].set_title('RGB', y=-.1)
    
    # dNBR imagery
    value1 = np.mean(bs)
    mean1 = value1.item()
    bs.plot(ax=ax[1], 
            cmap = 'gray',
            vmax = 1,
            vmin = -1)
    ax[1].set_title(f'Basin dNBR: {mean1:.3g}', y=-.1)
    ax[1].axis('off')
    
    # PPP imagery
    value2 = np.mean(prob)
    mean2 = value2.item()
    prob.plot(ax=ax[2], 
            cmap = 'gnuplot',
            vmax = 1,
            vmin = 0)
    ax[2].set_title(f'Basin P: {mean2:.3g}', y=-.1)
    ax[2].axis('off')
    if savefig == True:
       plt.savefig(os.path.join(fig_out, f'{basin}_{int(val*4)}mmhr_vis_1by3.png'))
    plt.show()
    return fig

#####



def fig_1x4(tci,
             tci2,
             bs, 
             prob,
             savefig=False):
    # Create figures
    fig, ax = plt.subplots(1, 4, 
                           figsize=(18,4), 
                           gridspec_kw={'width_ratios':[1, 1, 1.2, 1.2]})
    fig.suptitle(f'Basin {basin} with {int(val*4)}mm/hr Design Storm', fontsize=15, fontweight='bold')
    
    # RGB imagery
    tci.plot.imshow(ax=ax[0])
    ax[0].axis('off')
    ax[0].set_title('Pre-fire', y=-.1)
    
    tci2.plot.imshow(ax=ax[1])
    ax[1].axis('off')
    ax[1].set_title('Post-fire', y=-.1)
    
    # dNBR imagery
    value1 = np.mean(bs)
    mean1 = value1.item()
    bs.plot(ax=ax[2], 
            cmap = 'gray',
            vmax = 1,
            vmin = -1)
    ax[2].set_title(f'Basin dNBR: {mean1:.3g}', y=-.1)
    ax[2].axis('off')
    
    # PPP imagery
    value2 = np.mean(prob)
    mean2 = value2.item()
    prob.plot(ax=ax[3], 
            cmap = 'gnuplot',
            vmax = 1,
            vmin = 0)
    ax[3].set_title(f'Basin P: {mean2:.3g}', y=-.1)
    ax[3].axis('off')
    if savefig == True:
       plt.savefig(os.path.join(fig_out, f'{basin}_{int(val*4)}mmhr_vis_by4.png'))
    plt.show()
    return fig

def fig_6(tci, tci2, dnbr, ppp, p, savefig=False):
     individual_basin = x_values_df.index[x_values_df['basin_id'] == int(basin)]  
     usgs_x2 = float(x_values_df['x2'].iloc[individual_basin])
     value1 = np.mean(dnbr)
     mean1 = value1.item()
     value2 = np.mean(ppp)
     mean2 = value2.item()
     vals, counts = np.unique(dnbr, return_counts=True)
     mode_value = np.argwhere(counts == np.max(counts))
     fig = plt.figure(figsize = (15,8))
     gs = gridspec.GridSpec(2, 3, 
                            width_ratios=[1,1.2,1.2])
     fig.suptitle(f'Basin {basin}, {int(val*4)}mm/hr Design Storm', fontsize=15, fontweight='bold')
    
     # Create axes for top three images
     ax1 = fig.add_subplot(gs[0, 0])
     ax2 = fig.add_subplot(gs[0, 1])
     ax3 = fig.add_subplot(gs[0, 2])

     # Create axes for bottom two images
     ax4 = fig.add_subplot(gs[1, 0])
     ax5 = fig.add_subplot(gs[1, 1])
     ax6 = fig.add_subplot(gs[1, 2])
     
     fig.subplots_adjust(wspace=.25)
     # Plot images on axes
     tci.plot.imshow(ax=ax1)
     ax1.axis('off')
     ax1.set_title('Pre-fire', y=-.1)
     
     dnbr.plot(ax=ax2, 
            cmap = 'gray',
            vmax = 1,
            vmin = -1)
     ax2.set_title(f'Basin dNBR: {mean1:.3g}', y=-.1)
     ax2.axis('off')
     
     ppp.plot(ax=ax3, 
            cmap = 'gnuplot',
            vmax = 1,
            vmin = 0)
     ax3.set_title(f'Basin P: {mean2:.3g}', y=-.1)
     ax3.axis('off')
     
     tci2.plot.imshow(ax=ax4)
     ax4.axis('off')
     ax4.set_title('Post-fire', y=-.17)   
    
     dnbr.plot.hist(
                    color= 'c',
                    edgecolor='k',
                    bins=20,
                    ax=ax5)
     ax5.set(
         title='dNBR Distribution',
         xlabel=f'USGS x2: {usgs_x2: .3g}, Study x2: {mean1: .3g}',
         ylabel='Frequency')
     ax5.axvline(mean1, 
                 color='r', 
                 linestyle='solid', 
                 linewidth=2,
                 )
     ax5.axvline(usgs_x2, 
                 color='y', 
                 linestyle='solid', 
                 linewidth=2,
                 )
     ax5.axvline(mode_value, 
                 color='k', 
                 linestyle='solid', 
                 linewidth=2,
                 )
     ax5.set_xlim([-1,1])
     ppp.plot.hist(
                    color='c',
                    edgecolor='k',
                    bins=20,
                    ax=ax6)
     
     ax6.axvline(mean2, 
                 color='r', 
                 linestyle='solid', 
                 linewidth=2,
                )
     ax6.axvline(p, 
                 color='y', 
                 linestyle='solid', 
                 linewidth=2,
                 )  
     ax6.set(
        title='Probability Distribution',
        xlabel=f'USGS P:{p: .3g}, PP avg: {mean2: .3g}')
     ax6.set_xlim([0,1])
     
     
     if savefig == True:
         plt.savefig(os.path.join(fig_out, f'{basin}_{int(val*4)}mmhr_by6.png'))   
     plt.show()
     #plt.close()
     return        
     
        
def p_choice(val):
    individual_basin = x_values_df.index[x_values_df['basin_id'] == int(basin)]
    if val == 3.0:
        p = float(x_values_df['P_3'].iloc[individual_basin])
    elif val == 4.0:
        p = float(x_values_df['P_4'].iloc[individual_basin])
    elif val == 5.0:
        p = float(x_values_df['P_5'].iloc[individual_basin])
    elif val == 6.0:
        p = float(x_values_df['P_6'].iloc[individual_basin])
    elif val == 7.0:
        p = float(x_values_df['P_7'].iloc[individual_basin])
    elif val == 8.0:
        p = float(x_values_df['P_8'].iloc[individual_basin])
    elif val == 9.0:
        p = float(x_values_df['P_9'].iloc[individual_basin])
    elif val == 10.0:
        p = float(x_values_df['P_10'].iloc[individual_basin])
    p_list.append(p)
    return 

  

def fig_1x2(dnbr, 
            ppp, R, p,
            savefig=False):
     fig, ax = plt.subplots(1, 2, 
                           figsize=(14,5))
     fig.suptitle(f'Basin {basin} Data Distribution with {int(R *4)}mm/hr Design Storm', fontsize=15, fontweight='bold')
     individual_basin = x_values_df.index[x_values_df['basin_id'] == int(basin)]  
     usgs_x2 = float(x_values_df['x2'].iloc[individual_basin])
     value1 = np.mean(dnbr)
     mean1 = value1.item()
     value2 = np.mean(ppp)
     mean2 = value2.item()
     # dNBR histogram
     ax1 = ax[0]
     ax2 = ax[1]
     fig.tight_layout()
     dnbr.plot.hist(
                    color= 'c',
                    edgecolor='k',
                    bins=20,
                    ax=ax1)
     ax1.set(
         title='dNBR Distribution',
         xlabel=f'A) USGS dNBR: {usgs_x2: .3g} , Study dNBR : {mean1: .3g}',
         ylabel='Frequency')
     ax1.set_xlim([-1,1])
     ax1.axvline(usgs_x2, 
                 color='b', 
                 linestyle='solid', 
                 linewidth=2,
                 label = 'A) USGS dNBR'
                 )
     ax1.axvline(mean1, 
                 color='r', 
                 linestyle='solid', 
                 linewidth=2,
                 label = 'B) Study dNBR'
                 )
     
     ax1.yaxis.set_label_position('right')
     ax1.yaxis.tick_right()
     ax1.legend(loc='best')
     sns.histplot(dnbr, color='red', 
                  kde=True, ax=ax1)
     
     
     value2 = np.mean(ppp)
     pppavg = value2.item()
     ppp.plot.hist(
                    color='c',
                    edgecolor='k',
                    bins=bin_val,
                    ax=ax2)
     ax2.axvline(p, 
                 color='b', 
                 linestyle='solid', 
                 linewidth=2,
                 label = 'C) USGS P'
                 ) 
     ax2.axvline(mean2, 
                 color='r', 
                 linestyle='solid', 
                 linewidth=2,
                 label = 'D) Study P'
                )
      
     ax2.set(
        title='Probability Distribution',
        xlabel=f'USGS P: {p: .3g}, PP avg: {mean2: .3g}',
        ylabel='Frequency')
     ax2.yaxis.set_label_position('right')
     ax2.yaxis.tick_right()
     ax2.set_xlim([0,1])
     #ax2.set_ylim(0, y_max)
     ax2.legend(loc='best')
        
     if savefig == True:
        plt.savefig(os.path.join(fig_out, f'{basin}_{int(R *4)}mmhr_hist.png'))
     plt.show()
     return 

def fig_8(tci, tci2, avg_dnbr, dnbr, pha, ppp, pha_mode, p, R, shape, mode, savefig=False):
     individual_basin = x_values_df.index[x_values_df['basin_id'] == int(basin)]  
     usgs_x2 = float(x_values_df['x2'].iloc[individual_basin])
     value1 = np.mean(dnbr)
     mean1 = value1.item()
     value2 = np.mean(ppp)
     mean2 = value2.item()
     value3 = np.nanmedian(dnbr)
     median = value3.item()
     value4 = np.nanmedian(ppp)
     median2 = value4.item()
     value5 = np.mean(pha_mode)
     jalalala = value5.item()
     print(mode)
     fig, ax = plt.subplots(2, 4, 
                           figsize=(18,10), 
                           gridspec_kw={'width_ratios':[1, 1.2, 1.2, 1.8]})
     plt.subplots_adjust(right=.9)
     fig.suptitle(f'Basin {basin}, {int(R *4)}mm/hr Design Storm', 
                  fontsize=15, fontweight='bold')
     ax1 = ax[0,0] 
     ax2 = ax[0,1]
     ax3 = ax[0,2]
     ax4 = ax[0,3]
     ax5 = ax[1,0]
     ax6 = ax[1,1]
     ax7 = ax[1,2]
     ax8 = ax[1,3]
     
     # Plot images on axes
     fig.tight_layout(pad=5)
     
     tci.plot.imshow(ax=ax1)
     shape.plot(ax = ax1, alpha=0) 
     ax1.axis('off')
     ax1.set_title('Pre-fire')
     shape.plot(ax = ax2, alpha=0)   
     avg_dnbr.plot(ax=ax2, 
               cmap = 'gray',
               vmax = 1,
               vmin = -1)
     ax2.set_title(f'A) USGS dNBR mean: {usgs_x2: .3g}')
     ax2.axis('off')
     shape.plot(ax = ax3, alpha=0) 
     
     dnbr.plot(ax=ax3, 
            cmap = 'gray',
            vmax = 1,
            vmin = -1)
     ax3.set_title(f'B) Study dNBR mean: {mean1:.3g}')
     ax3.axis('off')
     
     dnbr.plot.hist(
                    color= 'c',
                    edgecolor='k',
                    bins=20,
                    ax=ax4)
     ax4.set(
         title=f'dNBR Distribution, median: {median: .3g}, Peak F: {mode: .3g}',
         #xlabel=f'USGS x2: , Study x2: {mean1: .3g}',
         ylabel='Frequency')
     ax4.set_xlim([-1,1])
     ax4.axvline(usgs_x2, 
                 color='b', 
                 linestyle='solid', 
                 linewidth=2,
                 label = 'A) USGS dNBR'
                 )
     ax4.axvline(mean1, 
                 color='r', 
                 linestyle='solid', 
                 linewidth=2,
                 label = 'B) Study dNBR'
                 )
     ax4.axvline(median, 
                 color='g', 
                 linestyle='solid', 
                 linewidth=2,
                 label = 'Median'
                 )
     ax4.axvline(mode, 
                 color='y', 
                 linestyle='solid', 
                 linewidth=2,
                 label = 'Peak F'
                 )
     
     ax4.yaxis.set_label_position('right')
     ax4.yaxis.tick_right()
     ax4.legend(loc='best')
        
     tci2.plot.imshow(ax=ax5)
     shape.plot(ax = ax5, alpha=0) 
     ax5.axis('off')
     ax5.set_title('Post-fire')  
     shape.plot(ax = ax6, alpha=0) 
     pha.plot(ax=ax6, 
            cmap = 'gnuplot',
            vmax = 1,
             vmin = 0)
     ax6.set_title(f'C) USGS P:{p: .3g}')
     ax6.axis('off')
     shape.plot(ax = ax7, alpha=0) 
     ppp.plot(ax=ax7, 
            cmap = 'gnuplot',
            vmax = 1,
            vmin = 0)
     ax7.set_title(f'D) Pixel P mean: {mean2:.3g}')
     ax7.axis('off')
     
     ppp.plot.hist(
                    color='c',
                    edgecolor='k',
                    bins=bin_val,
                    ax=ax8)
     ax8.axvline(p, 
                 color='b', 
                 linestyle='solid', 
                 linewidth=2,
                 label = 'C) USGS P'
                 ) 
     ax8.axvline(mean2, 
                 color='r', 
                 linestyle='solid', 
                 linewidth=2,
                 label = 'D) Study P'
                )
     ax8.axvline(median2, 
                 color='g', 
                 linestyle='solid', 
                 linewidth=2,
                 label = 'Median'
                 )
     ax8.axvline(jalalala, 
                 color='y', 
                 linestyle='solid', 
                 linewidth=2,
                 label = 'Peak F'
                 )
     
     ax8.set(
        title=f'Probability Distribution, median: {median2: .3g}, Peak F: {jalalala: .3g}',
        #xlabel=f'PP avg: {mean2: .3g}',
        ylabel='Frequency')
     ax8.yaxis.set_label_position('right')
     ax8.yaxis.tick_right()
     ax8.set_xlim([0,1])
     ax8.set_ylim(0, y_max)
     ax8.legend(loc='best')
     
     if savefig == True:
         plt.savefig(os.path.join(fig_out, f'{basin}_{int(R *4)}mmhr_all_2x4.png'))   
     plt.show()
     #plt.close()
     return  
def fig_sr(avg_dnbr, dnbr, pha_12, pha_24, pha_36, ppp_12, ppp_24, ppp_36,
          p_12, p_24, p_36, shape, savefig=False):
     individual_basin = x_values_df.index[x_values_df['basin_id'] == int(basin)]  
     usgs_x2 = float(x_values_df['x2'].iloc[individual_basin])
     value1 = np.mean(dnbr)
     mean1 = value1.item()
     value2 = np.mean(ppp_12)
     mean2 = value2.item()
     value3 = np.mean(ppp_24)
     mean3 = value3.item()
     value4 = np.mean(ppp_36)
     mean4 = value4.item()
     
     fig, ax = plt.subplots(2, 4, 
                           figsize=(18,10))
     
     fig.suptitle(f'Basin {basin}, Whole Basin vs Per-Pixel Probability', 
                  fontsize=15, fontweight='bold')
     ax1 = ax[0,0] 
     ax2 = ax[0,1]
     ax3 = ax[0,2]
     ax4 = ax[0,3]
     ax5 = ax[1,0]
     ax6 = ax[1,1]
     ax7 = ax[1,2]
     ax8 = ax[1,3]
     
     # Plot images on axes
     fig.tight_layout(pad=2)
     
     shape.plot(ax = ax1, alpha=0)   
     avg_dnbr.plot(ax=ax1, 
               cmap = 'gray',
               vmax = 1,
               vmin = -1)
     ax1.set_title(f'USGS dNBR mean: {usgs_x2: .3g}')
     ax1.axis('off')
     
     shape.plot(ax = ax2, alpha=0) 
     pha_12.plot(ax=ax2, 
            cmap = 'gnuplot',
            vmax = 1,
             vmin = 0)
     ax2.set_title(f'12mm/hr USGS P:{p_12: .3g}')
     ax2.axis('off')
    
     shape.plot(ax = ax3, alpha=0) 
     pha_24.plot(ax=ax3, 
            cmap = 'gnuplot',
            vmax = 1,
             vmin = 0)
     ax3.set_title(f'24mm/hr USGS P:{p_24: .3g}')
     ax3.axis('off')
    
     shape.plot(ax = ax4, alpha=0) 
     pha_36.plot(ax=ax4, 
            cmap = 'gnuplot',
            vmax = 1,
             vmin = 0)
     ax4.set_title(f'36mm/hr USGS P:{p_36: .3g}')
     ax4.axis('off')
    
     shape.plot(ax = ax5, alpha=0) 
     dnbr.plot(ax=ax5, 
            cmap = 'gray',
            vmax = 1,
            vmin = -1)
     ax5.set_title(f'Study dNBR mean: {mean1:.3g}')
     ax5.axis('off')
         
     shape.plot(ax = ax6, alpha=0) 
     ppp_12.plot(ax=ax6, 
            cmap = 'gnuplot',
            vmax = 1,
             vmin = 0)
     ax6.set_title(f'12mm/hr Study P avg:{mean2: .3g}')
     ax6.axis('off')
    
     shape.plot(ax = ax7, alpha=0) 
     ppp_24.plot(ax=ax7, 
            cmap = 'gnuplot',
            vmax = 1,
             vmin = 0)
     ax7.set_title(f'24mm/hr Study P avg:{mean3: .3g}')
     ax7.axis('off')
    
     shape.plot(ax = ax8, alpha=0) 
     ppp_36.plot(ax=ax8, 
            cmap = 'gnuplot',
            vmax = 1,
             vmin = 0)
     ax8.set_title(f'36mm/hr Study P avg:{mean4: .3g}')
     ax8.axis('off')
     
     if savefig == True:
         plt.savefig(os.path.join(fig_out, f'{basin}_prediction_comparison.png'))   
     plt.show()
     #plt.close()
     return  

def ppp_hist(one, two, three, four, savefig=False):    
    fig, ax = plt.subplots(1, 4, 
                           figsize=(25,5), 
                           sharey=True)
     
    fig.suptitle(f"Basin {basin} Per Pixel Probability Distribution", 
                  fontsize=15, fontweight='bold')
    ax1 = ax[0] 
    ax2 = ax[1]
    ax3 = ax[2]
    ax4 = ax[3]
    fig.tight_layout()
    one.plot.hist(
                  bins = bin_val,
                  label = '12mm/hr',
                  color = 'c',
                  edgecolor = 'k', ax=ax1)
    ax1.set(title = '12mm/hr',
           ylabel = 'Frequency')
    ax1.set_ylim(0, y_max)
    two.plot.hist(
                  bins = bin_val,
                  label = '12mm/hr',
                  color = 'c',
                  edgecolor = 'k', ax=ax2)
    ax2.set(title = '20mm/hr')
    three.plot.hist(
                  bins = bin_val,
                  label = '12mm/hr',
                  color = 'c',
                  edgecolor = 'k', ax=ax3)
    ax3.set(title = '28mm/hr')
    four.plot.hist(
                  bins = bin_val,
                  label = '12mm/hr',
                  color = 'c',
                  edgecolor = 'k', ax=ax4)
    ax4.set(title = '36mm/hr')
    if savefig == True:
         plt.savefig(os.path.join(fig_out, f'{basin}_rchange.png'))   
    
    plt.show()
    return
def compare(avg_dnbr, pha, pha_median, pha_mode, p, R, savefig=False):
    individual_basin = x_values_df.index[x_values_df['basin_id'] == int(basin)]  
    usgs_x2 = float(x_values_df['x2'].iloc[individual_basin])
    value1 = np.mean(pha_mode)
    mode_p = value1.item()
    value2 = np.mean(pha_median)
    median_p = value2.item()
    fig, ax = plt.subplots(1, 4, 
                           figsize=(15,5))
    
    fig.suptitle(f'Basin {basin}, {int(R*4)}mm/hr PHA Comparison', 
                  fontsize=15, fontweight='bold')
    fig.tight_layout(pad=3)
    
    ax1 = ax[0] 
    ax2 = ax[1]
    ax3 = ax[2]
    ax4 = ax[3]
    shape.plot(ax = ax1, alpha=0)   
    avg_dnbr.plot(ax=ax1, 
               cmap = 'gray',
               vmax = 1,
               vmin = -1)
    ax1.set_title(f'USGS dNBR mean: {usgs_x2: .3g}')
    ax1.axis('off')
    
    shape.plot(ax = ax2, alpha=0) 
    pha.plot(ax=ax2, 
            cmap = 'gnuplot',
            vmax = 1,
             vmin = 0)
    ax2.set_title(f'USGS P:{p: .3g}')
    ax2.axis('off')
    
    shape.plot(ax = ax3, alpha=0) 
    pha_median.plot(ax=ax3, 
            cmap = 'gnuplot',
            vmax = 1,
             vmin = 0)
    ax3.set_title(f'Median P:{median_p: .3g}')
    ax3.axis('off')
    
    shape.plot(ax = ax4, alpha=0) 
    pha_mode.plot(ax=ax4, 
            cmap = 'gnuplot',
            vmax = 1,
             vmin = 0)
    ax4.set_title(f'Peak F P:{mode_p: .3g}')
    ax4.axis('off')
    
    if savefig == True:
         plt.savefig(os.path.join(fig_out, f'{basin}_{int(R*4)}mmhr_model_comparison.png'))   
    
    plt.show()
    return

def compare_ppp(avg_dnbr, pha, pha_median, pha_mode, ppp, p, R, savefig=False):
    individual_basin = x_values_df.index[x_values_df['basin_id'] == int(basin)]  
    usgs_x2 = float(x_values_df['x2'].iloc[individual_basin])
    value1 = np.mean(pha_mode)
    mode_p = value1.item()
    value2 = np.mean(pha_median)
    median_p = value2.item()
    fig, ax = plt.subplots(1, 5, 
                           figsize=(20,5))
    
    fig.suptitle(f'Basin {basin}, {int(R*4)}mm/hr PHA & PPP Comparison', 
                  fontsize=15, fontweight='bold')
    fig.tight_layout(pad=2)
    
    ax1 = ax[0] 
    ax2 = ax[1]
    ax3 = ax[2]
    ax4 = ax[3]
    ax5 = ax[4]
    shape.plot(ax = ax1, alpha=0)   
    avg_dnbr.plot(ax=ax1, 
               cmap = 'gray',
               vmax = 1,
               vmin = -1)
    ax1.set_title(f'USGS dNBR mean: {usgs_x2: .3g}')
    ax1.axis('off')
    
    shape.plot(ax = ax2, alpha=0) 
    pha.plot(ax=ax2, 
            cmap = 'gnuplot',
            vmax = 1,
             vmin = 0)
    ax2.set_title(f'USGS P:{p: .3g}')
    ax2.axis('off')
    
    shape.plot(ax = ax3, alpha=0) 
    pha_median.plot(ax=ax3, 
            cmap = 'gnuplot',
            vmax = 1,
             vmin = 0)
    ax3.set_title(f'Median P:{median_p: .3g}')
    ax3.axis('off')
    
    shape.plot(ax = ax4, alpha=0) 
    pha_mode.plot(ax=ax4, 
            cmap = 'gnuplot',
            vmax = 1,
             vmin = 0)
    ax4.set_title(f'Mode P:{mode_p: .3g}')
    ax4.axis('off')
    
    shape.plot(ax = ax5, alpha=0) 
    ppp.plot(ax=ax5, 
            cmap = 'gnuplot',
            vmax = 1,
             vmin = 0)
    ax5.set_title(f'Location')
    ax5.axis('off')
    
    if savefig == True:
         plt.savefig(os.path.join(fig_out, f'{basin}_{int(R*4)}mmhr_p_loc.png'))   
    
    plt.show()
    return

def compare_mh(dnbrm, dnbrh, pppm, ppph, R, savefig=False):
    individual_basin = x_values_df.index[x_values_df['basin_id'] == int(basin)]  
    value1 = np.mean(dnbrm)
    mean1 = value1.item()
    value2 = np.mean(dnbrh)
    mean2 = value2.item()
    fig, ax = plt.subplots(2, 3, 
                           figsize=(14,8))
    
    fig.suptitle(f'Basin {basin} Sentinel-2b vs AVIRIS, {int(R*4)}mmhr', 
                  fontsize=15, fontweight='bold')
    fig.tight_layout(pad = 4)
    
    ax1 = ax[0,0] 
    ax2 = ax[0,1]
    ax3 = ax[0,2]
    ax4 = ax[1,0]
    ax5 = ax[1,1]
    ax6 = ax[1,2]
    ##
    shape.plot(ax = ax1, alpha=0)   
    dnbrm.plot(ax=ax1, 
               cmap = 'gray',
               vmax = 1.5,
               vmin = -1.5)
    ax1.set_title(f'Sentinel dNBR avg: {mean1: .3g}')
    ax1.axis('off')
    
    shape.plot(ax = ax2, alpha=0) 
    dnbrh.plot(ax=ax2, 
            cmap = 'gray',
            vmax = 1.5,
             vmin = -1.5)
    ax2.set_title(f'AVIRIS dNBR avg:{mean2: .3g}')
    ax2.axis('off')
    
    
    dnbrm.plot.hist(
                  bins = dnbr_bin_val,
                  color = 'c',
                  alpha = .5, label = 'Sentinel',
                  edgecolor = 'k', ax=ax3)
    dnbrh.plot.hist(
                  bins = dnbr_bin_val,
                  color = 'r',
                  alpha = .5, label = 'AVIRIS',
                  edgecolor = 'k', ax=ax3)
    ax3.yaxis.set_label_position('right')
    ax3.yaxis.tick_right()
    ax3.set(title = 'dNBR distribution')
    ax3.set(ylabel = 'Frequency')
    ax3.legend(loc='best')
    ##
    
    
    shape.plot(ax = ax4, alpha=0) 
    pppm.plot(ax=ax4, 
            cmap = 'gnuplot',
            vmax = 1,
             vmin = 0)
    ax4.set_title(f'Sentinel Pixel Probability')
    ax4.axis('off')
    shape.plot(ax = ax5, alpha=0) 
    ppph.plot(ax=ax5, 
            cmap = 'gnuplot',
            vmax = 1,
             vmin = 0)
    ax5.set_title(f'AVIRIS Pixel Probability')
    ax5.axis('off')
    pppm.plot.hist(
                  bins = bin_val,
                  color = 'c',
                  alpha = .5, label = 'Sentinel',
                  edgecolor = 'k', ax=ax6)
    ppph.plot.hist(
                  bins = bin_val,
                  color = 'r',
                  alpha = .5, label = 'AVIRIS',
                  edgecolor = 'k', ax=ax6)
    ax6.set(title = 'Probability Distribution')
    ax6.yaxis.set_label_position('right')
    ax6.yaxis.tick_right()
    ax6.set(ylabel = 'Frequency')
    ax6.set_xlim([0,1])
    ax6.set_ylim(0, y_max)
    ax6.legend(loc='best')
    
    if savefig == True:
         plt.savefig(os.path.join(fig_out, f'{basin}_{int(R*4)}_SentvsAV.png'))   
    
    plt.show()
    return



def compare_nbr(mnbr_pre, mnbr_post, hnbr_pre, hnbr_post, savefig=False):
    individual_basin = x_values_df.index[x_values_df['basin_id'] == int(basin)]  
    value1 = np.mean(mnbr_pre)
    mean1 = value1.item()
    value2 = np.mean(mnbr_post)
    mean2 = value2.item()
    value3 = np.mean(hnbr_pre)
    mean3 = value3.item()
    value4 = np.mean(hnbr_post)
    mean4 = value4.item()
    fig, ax = plt.subplots(2, 3, 
                           figsize=(14,8))
    
    fig.suptitle(f'Basin {basin} Sentinel-2b vs AVIRIS NBR', 
                  fontsize=15, fontweight='bold')
    fig.tight_layout(pad = 3)
    
    ax1 = ax[0,0] 
    ax2 = ax[0,1]
    ax3 = ax[0,2]
    ax4 = ax[1,0]
    ax5 = ax[1,1]
    ax6 = ax[1,2]
    ##
    shape.plot(ax = ax1, alpha=0)   
    mnbr_pre.plot(ax=ax1, 
               cmap = 'turbo_r',
               vmax = 1,
               vmin = -1)
    ax1.set_title(f'Sentinel Prefire NBR avg: {mean1: .3g}')
    ax1.axis('off')
    
    shape.plot(ax = ax2, alpha=0) 
    mnbr_post.plot(ax=ax2, 
            cmap = 'turbo_r',
            vmax = 1,
             vmin = -1)
    ax2.set_title(f'Sentinel Postfire NBR avg:{mean2: .3g}')
    ax2.axis('off')
    
    
    mnbr_pre.plot.hist(
                  bins = nbr_bin_val,
                  color = 'c',
                  alpha = .5, label = 'Prefire',
                  edgecolor = 'k', ax=ax3)
    mnbr_post.plot.hist(
                  bins = nbr_bin_val,
                  color = 'r',
                  alpha = .5, label = 'Postfire',
                  edgecolor = 'k', ax=ax3)
    ax3.yaxis.set_label_position('right')
    ax3.yaxis.tick_right()
    ax3.set(title = 'Sentinel NBR distribution')
    ax3.legend(loc='best')
    ##
    
    
    shape.plot(ax = ax4, alpha=0) 
    hnbr_pre.plot(ax=ax4, 
            cmap = 'turbo_r',
            vmax = 1,
             vmin = -1)
    ax4.set_title(f'AVIRIS Prefire NBR avg: {mean3: .3g}')
    ax4.axis('off')
    shape.plot(ax = ax5, alpha=0) 
    hnbr_post.plot(ax=ax5, 
            cmap = 'turbo_r',
            vmax = 1,
             vmin = -1)
    ax5.set_title(f'AVIRIS Postfire NBR avg: {mean4: .3g}')
    ax5.axis('off')
    hnbr_pre.plot.hist(
                  bins = nbr_bin_val,
                  color = 'c',
                  alpha = .5, label = 'Prefire',
                  edgecolor = 'k', ax=ax6)
    hnbr_post.plot.hist(
                  bins = nbr_bin_val,
                  color = 'r',
                  alpha = .5, label = 'Postfire',
                  edgecolor = 'k', ax=ax6)
    ax6.set(title = 'AVIRIS NBR Distribution')
    ax6.yaxis.set_label_position('right')
    ax6.yaxis.tick_right()
    ax6.legend(loc='best')
    
    if savefig == True:
         plt.savefig(os.path.join(fig_out, f'{basin}_nbr_SentvsAV.png'))   
    
    plt.show()
    return

def compare_nbr2(mnbr_pre, mnbr_post, hnbr_pre, hnbr_post, savefig=False):
    individual_basin = x_values_df.index[x_values_df['basin_id'] == int(basin)]  
    value1 = np.mean(mnbr_pre)
    mean1 = value1.item()
    value2 = np.mean(mnbr_post)
    mean2 = value2.item()
    value3 = np.mean(hnbr_pre)
    mean3 = value3.item()
    value4 = np.mean(hnbr_post)
    mean4 = value4.item()
    fig, ax = plt.subplots(2, 3, 
                           figsize=(14,8))
    
    fig.suptitle(f'Basin {basin} Sentinel-2b vs AVIRIS NBR', 
                  fontsize=15, fontweight='bold')
    fig.tight_layout(pad = 4)
    
    ax1 = ax[0,0] 
    ax2 = ax[0,1]
    ax3 = ax[0,2]
    ax4 = ax[1,0]
    ax5 = ax[1,1]
    ax6 = ax[1,2]
    ##
    shape.plot(ax = ax1, alpha=0)   
    mnbr_pre.plot(ax=ax1, 
               cmap = 'turbo_r',
               vmax = 1,
               vmin = -1)
    ax1.set_title(f'Sentinel Prefire NBR avg: {mean1: .3g}')
    ax1.axis('off')
    
    shape.plot(ax = ax2, alpha=0) 
    hnbr_pre.plot(ax=ax2, 
            cmap = 'turbo_r',
            vmax = 1,
             vmin = -1)
    ax2.set_title(f'AVIRIS Prefire NBR avg: {mean3: .3g}')
    ax2.axis('off')
    
    mnbr_pre.plot.hist(
                  bins = nbr_bin_val,
                  color = 'c',
                  alpha = .5, label = 'Sentinel',
                  edgecolor = 'k', ax=ax3)
    hnbr_pre.plot.hist(
                  bins = nbr_bin_val,
                  color = 'r',
                  alpha = .5, label = 'AVIRIS',
                  edgecolor = 'k', ax=ax3)
    ax3.yaxis.set_label_position('right')
    ax3.yaxis.tick_right()
    ax3.set(ylabel = 'Frequency')
    ax3.set(title = 'Prefire NBR distribution')
    ax3.legend(loc='best')
    
    ##
    
    shape.plot(ax = ax4, alpha=0) 
    mnbr_post.plot(ax=ax4, 
            cmap = 'turbo_r',
            vmax = 1,
             vmin = -1)
    ax4.set_title(f'Sentinel Postfire NBR avg:{mean2: .3g}')
    ax4.axis('off')
    shape.plot(ax = ax5, alpha=0) 
    hnbr_post.plot(ax=ax5, 
            cmap = 'turbo_r',
            vmax = 1,
             vmin = -1)
    ax5.set_title(f'AVIRIS Postfire NBR avg: {mean4: .3g}')
    ax5.axis('off')
    mnbr_post.plot.hist(
                  bins = nbr_bin_val,
                  color = 'c',
                  alpha = .5, label = 'Sentinel',
                  edgecolor = 'k', ax=ax6)
    hnbr_post.plot.hist(
                  bins = nbr_bin_val,
                  color = 'r',
                  alpha = .5, label = 'AVIRIS',
                  edgecolor = 'k', ax=ax6)
    ax6.set(title = 'Postfire NBR Distribution')
    ax6.yaxis.set_label_position('right')
    ax6.set(ylabel = 'Frequency')
    ax6.yaxis.tick_right()
    ax6.legend(loc='best')
    
    if savefig == True:
         plt.savefig(os.path.join(fig_out, f'{basin}_nbr_dir.png'))   
    
    plt.show()
    return

In [115]:
from scipy.signal import argrelextrema
choice = input('''Input your choice.
                  1: Run all basins with R value = 24 mm/hr.
                  2: Run all basins with all R values.
                  ''')
x_values_df = pd.read_csv('/mnt/nfs/lss/meerdink/home/skzebarth/masters/basin_data/ma_data.csv', 
                          delimiter=',')
hyper_dnbr = rxr.open_rasterio(av_dnbr, masked=True)
nbr_bin_val = [-1, -.9, -.8, -.7, -.6, -.5, -.4, -.3, -.2, -.1, 0, .1, .2, .3, 
                            .4, .5, .6, .7, .8, .9, 1]
dnbr_bin_val = [-1, -.9, -.8, -.7, -.6, -.5, -.4, -.3, -.2, -.1, 0, .1, .2, .3, 
                            .4, .5, .6, .7, .8, .9, 1, 1.1, 1.2, 1.3, 1.4, 1.5]
for filename in sorted(os.listdir(dirPolyLoc)):
    if filename.endswith('.shp'):
        b_name = filename.split('.')
        basin = b_name[0]
        print('done')
        break
        shape = gpd.read_file(os.path.join(dirPolyLoc, filename))
        av_dnbr_clip = hyper_dnbr.rio.clip(shape.geometry.values,
                                shape.crs,
                                all_touched = False,
                                drop = True,
                                invert = False)
        #av_dnbr_clip.plot()
        
        rgb_img = rgb(dirImgLoc, shape, plot=False, save=False)
        rgb_img2 = rgb(dirImgLocPost, shape, plot=False, save=False)
        pre_fire_img = nbr_clip(dirImgLoc, shape, plot=False, save=False)
        print(f'Prefire imagery and basins NBR analysis complete for basin {basin}.')
        post_fire_img = nbr_clip(dirImgLocPost, shape, plot=False, save=False)
        print(f'Postfire imagery and basins NBR analysis complete for basin {basin}.')
        
        h_pre_fire_img = av_nbr_clip(pre_h, shape, save=False)
        
        h_post_fire_img = av_nbr_clip(post_h, shape, save=False)
        
        test2 = compare_nbr2(pre_fire_img, post_fire_img, 
                             h_pre_fire_img, h_post_fire_img, savefig=True)
        
        
        dnbr_val = []
        
        avg_dnbr_img = avg_dnbr(pre_fire_img, post_fire_img, plot=False, save=False)
        dnbr_img = dnbr(pre_fire_img, post_fire_img, plot=False, save=False)
        y = np.round(dnbr_val, decimals = 3)
        y_flat = y.flatten()
#         #print(dnbr_val)
        
        ad = np.array(dnbr_val)
#         #print(ad)
        ax = ad.flatten()
#         #print(ax)
        #mode1 = scipy.stats.mode(y_flat, keepdims=True)
        #mode = mode1[0]
        mode2 = Counter(ax)
        print(mode2.most_common(1))
        mode3 = scipy.signal.find_peaks(ax, height=0)
        #modelehoo = np.argmax(mode3)
        #if 
        
        #print(mode3)
        hist, bins = np.histogram(ax, bins=50)
        peak_bin_index = np.argmax(hist)
        print(peak_bin_index)
        mode = bins[peak_bin_index]
        print(f'peak frequency should be {mode} partner')
        
        
#        print(f'the mode is {mode}')
        median = np.nanmedian(dnbr_img)
        ntest = scipy.stats.normaltest(ax)
        ntestsw = scipy.stats.shapiro(ax)
        print(ntest)
        print(ntestsw)
        
#        print(ntest)
#         print(ntestsw)
        
        
#         print(f'DNBR analysis complete for basin {basin}.')
        if choice == '1':
            R = [6.0]
            for val in R:
                p = p_choice(val)
                pha_img = pha(dnbr_img, plot=False, save=False)
                ppp_img = ppp(dnbr_img, plot=False, save=False)
                
                print(f'Per Pixel Analysis complete for basin {basin}.')
                #fig = fig_1x3(rgb_img, dnbr_img, ppp_img, savefig=False)
                
#                 #fig2 = fig_1x4(rgb_img, rgb_img2, dnbr_img, ppp_img, savefig=False)
#                 #fig1 = fig_6(rgb_img, rgb_img2, dnbr_img, ppp_img, p, savefig=False)
#                 fig = fig_8(rgb_img, rgb_img2, avg_dnbr_img, dnbr_img,
#                             pha_img, ppp_img, p, savefig=False)
                
#             #print(f'Basin {basin} Debris Flow Likelihood analysis completed R value equicalent to 24mm/hr')
        if choice == '2':
            R = [3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
            bin_val = [0.0, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.55,
               0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95, 1.0]
            dnbr_bin_val = [-1, -.9, -.8, -.7, -.6, -.5, -.4, -.3, -.2, -.1, 0, .1, .2, .3, 
                            .4, .5, .6, .7, .8, .9, 1, 1.1, 1.2, 1.3, 1.4, 1.5]
            pha_list = []
            pha_mode_list = []
            pha_median_list = []
            ppp_list = []
            ppp_hyper_list = []
            dist_list = []
            p_list = []
            
            for val in R:
                p = p_choice(val)
                pha_img = pha(dnbr_img, plot=False, save=False)
                pha_mode_img = pha_mode(dnbr_img, plot=False, save=False)
                pha_median_img = pha_median(dnbr_img, plot=False, save=False)
                ppp_img = ppp(dnbr_img, plot=False, save=False)
                ppp_hyp = ppp_av(av_dnbr_clip)
                ppp_hyper_list.append(ppp_hyp)
                print(f'Per Pixel Analysis complete for basin {basin}, with a design storm of {int(val)*4} mm/hr.')
#                 #fig = fig_1x3(rgb_img, dnbr_img, ppp_img, savefig=True)
#                 #hist = fig_1x2(dnbr_img, ppp_img, savefig=False)
#                 #fig2 = fig_1x4(rgb_img, rgb_img2, dnbr_img, ppp_img, savefig=True)
#                 #fig1 = fig_6(rgb_img, rgb_img2, dnbr_img, ppp_img, p, savefig=True)
            
            
            
            bf = []
            for val in dist_list:
                test_bin = pd.cut(val, bin_val)
                x = max(test_bin.value_counts())
                bf.append(x)
            
            y_max = round(max(bf) +10)
            
            print(f'Basin {basin} Debris Flow Likelihood analysis completed with all R values')
            
            fig_list_df = pd.DataFrame({
                'R': R,
                'pha': pha_list,
                'ppp': ppp_list,
                'ppp_h': ppp_hyper_list,
                'pha_mode': pha_mode_list,
                'pha_median': pha_median_list,
                'p': p_list
            })
# #             pha_12 = fig_list_df.loc[0]['pha']
# #             pha_24 = fig_list_df.loc[3]['pha']
# #             pha_36 = fig_list_df.loc[6]['pha']
# #             ppp_12 = fig_list_df.loc[0]['ppp']
# #             ppp_20 = fig_list_df.loc[2]['ppp']
# #             ppp_24 = fig_list_df.loc[3]['ppp']
# #             ppp_28 = fig_list_df.loc[4]['ppp']
# #             ppp_36 = fig_list_df.loc[6]['ppp']
# #             p_12 = fig_list_df.loc[0]['p']
# #             p_24 = fig_list_df.loc[3]['p']
# #             p_36 = fig_list_df.loc[6]['p']
            
# #             hist = ppp_hist(ppp_12, ppp_20, ppp_28, ppp_36, savefig=True)
            #fig = fig_sr(avg_dnbr_img, dnbr_img, pha_12, pha_24, pha_36, ppp_12, ppp_24, ppp_36,
                         #p_12, p_24, p_36, shape, savefig=True)
#             #print(fig_list_df)
            for i in range(len(fig_list_df)):
                fig = compare_mh(dnbr_img, av_dnbr_clip, 
                                 fig_list_df.loc[i, 'ppp'], fig_list_df.loc[i, 'ppp_h'],
                                fig_list_df.loc[i, 'R'], savefig=True)
                fig = compare(avg_dnbr_img, fig_list_df.loc[i,'pha'], fig_list_df.loc[i,'pha_median'], 
                              fig_list_df.loc[i,'pha_mode'], fig_list_df.loc[i, 'p'], 
                              fig_list_df.loc[i, 'R'], savefig=True)
#                 fig2 = compare_ppp(avg_dnbr_img, fig_list_df.loc[i,'pha'], fig_list_df.loc[i,'pha_median'], 
#                               fig_list_df.loc[i,'pha_mode'], fig_list_df.loc[i, 'ppp'], fig_list_df.loc[i, 'p'], 
#                               fig_list_df.loc[i, 'R'], savefig=True)
                fig2 = fig_8(rgb_img, rgb_img2, avg_dnbr_img, dnbr_img,
                             fig_list_df.loc[i,'pha'], fig_list_df.loc[i, 'ppp'],
                             fig_list_df.loc[i,'pha_mode'],
                             fig_list_df.loc[i, 'p'], fig_list_df.loc[i, 'R'], 
                             shape, mode, savefig=True)
        #else:
            #print(f'Your input of {choice} is invalid. Quitting...')
           # break
                  

Input your choice.
                  1: Run all basins with R value = 24 mm/hr.
                  2: Run all basins with all R values.
                  2
done


## choice = input('''Input your choice.
                  1: Run all basins with R value = 24 mm/hr.
                  2: Run all basins with all R values.
                  ''')
x_values_df = pd.read_csv('/mnt/nfs/lss/meerdink/home/skzebarth/masters/basin_data/ma_data.csv', 
                          delimiter=',')
print(x_values_df)

#hyper_dnbr = rxr.open_rasterio(av_dnbr, masked=True)
nbr_bin_val = [-1, -.9, -.8, -.7, -.6, -.5, -.4, -.3, -.2, -.1, 0, .1, .2, .3, 
                            .4, .5, .6, .7, .8, .9, 1]
dp = []
sw = []
for filename in sorted(os.listdir(dirPolyLoc)):
    if filename.endswith('.shp'):
        b_name = filename.split('.')
        basin = b_name[0]
        
        shape = gpd.read_file(os.path.join(dirPolyLoc, filename))
        av_dnbr_clip = hyper_dnbr.rio.clip(shape.geometry.values,
                                shape.crs,
                                all_touched = False,
                                drop = True,
                                invert = False)
        #av_dnbr_clip.plot()
        
        #rgb_img = rgb(dirImgLoc, shape, plot=False, save=False)
        #rgb_img2 = rgb(dirImgLocPost, shape, plot=False, save=False)
        pre_fire_img = nbr_clip(dirImgLoc, shape, plot=False, save=False)
        print(f'Prefire imagery and basins NBR analysis complete for basin {basin}.')
        post_fire_img = nbr_clip(dirImgLocPost, shape, plot=False, save=False)
        print(f'Postfire imagery and basins NBR analysis complete for basin {basin}.')
        
        h_pre_fire_img = av_nbr_clip(pre_h, shape, save=False)
        
        h_post_fire_img = av_nbr_clip(post_h, shape, save=False)
        
        test2 = compare_nbr2(pre_fire_img, post_fire_img, 
                             h_pre_fire_img, h_post_fire_img, savefig=False)
        
        
        dnbr_val = []
        
        avg_dnbr_img = avg_dnbr(pre_fire_img, post_fire_img, plot=False, save=False)
        dnbr_img = dnbr(pre_fire_img, post_fire_img, plot=False, save=False)
        y = np.round(dnbr_val, decimals = 3)
        y_flat = y.flatten()
#         #print(dnbr_val)
        ad = np.array(dnbr_val)
#         #print(ad)
        ax = ad.flatten()
#         #print(ax)
        mode1 = scipy.stats.mode(y_flat, keepdims=True)
        mode = mode1[0]
#        print(f'the mode is {mode}')
        median = np.nanmedian(dnbr_img)
        ntest = scipy.stats.normaltest(ax)
        ntestsw = scipy.stats.shapiro(ax)
        
        dp.append(ntest)
        
        sw.append(ntestsw)
        print(dp)
        print(sw)

        
        

In [29]:

dp_df = pd.DataFrame(dp, columns = ['Statistic', 'P'])
print(dp_df)
dp_df.to_csv(general+'dp.csv', index=False)

sw_df = pd.DataFrame(sw, columns = ['Statistic', 'P'])
print(sw_df)
sw_df.to_csv(general+'sw.csv', index=False)
# normal_tests = pd.DataFrame({
#     'DAgostino_Pearson' : [dp],
#     'Shapiro_Wilk' : [sw],
# })
# print(normal_tests)

#normal_tests.to_csv(general,'normal_tests.csv', index =False)

       Statistic              P
0     154.943792   2.261430e-34
1      75.622163   3.791891e-17
2     152.054074   9.591295e-34
3     264.323432   4.007707e-58
4   13175.063271   0.000000e+00
5     713.235289  1.327165e-155
6       6.339606   4.201188e-02
7       5.921988   5.176744e-02
8     117.494881   3.064155e-26
9    2088.317443   0.000000e+00
10   1573.624330   0.000000e+00
11     71.908797   2.427746e-16
12    379.030592   4.949403e-83
13     28.959526   5.146581e-07
14     24.065675   5.945729e-06
15    666.237912  2.129490e-145
16     19.176883   6.851612e-05
17     87.624604   9.387694e-20
18     71.842042   2.510145e-16
19    381.887723   1.186136e-83
20     32.666238   8.065228e-08
21     12.022209   2.451379e-03
22     14.837585   5.998729e-04
23    195.255814   3.987849e-43
24     83.722523   6.605188e-19
25      0.850135   6.537256e-01
26     18.776132   8.371720e-05
27     36.493432   1.190013e-08
28     34.472621   3.268628e-08
29     23.839506   6.657592e-06
30    35

## big_shape = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/basin_data/basin_shp/'
#print(os.listdir(big_shape))
for filename in sorted(os.listdir(big_shape)):
    if filename.endswith('ma_data_bool.shp'):
        #b_name = filename.split('.')
        basin = 'all_basins'
        
        shape = gpd.read_file(os.path.join(big_shape, filename))
        rgb_img = rgb(dirImgLoc, shape, plot=False, save=True)
        rgb_img2 = rgb(dirImgLocPost, shape, plot=False, save=True)
        pre_fire_img = nbr_clip(dirImgLoc, shape, plot=False, save=True)
        print(f'Prefire imagery and basins NBR analysis complete for basin .')
        post_fire_img = nbr_clip(dirImgLocPost, shape, plot=False, save=True)
        print(f'Postfire imagery and basins NBR analysis complete for basin .')
        #avg_dnbr_img = avg_dnbr(pre_fire_img, post_fire_img, plot=False, save=True)
        dnbr_img = dnbr(pre_fire_img, post_fire_img, plot=False, save=True)
        print(f'DNBR analysis complete for basin .')